In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sympy import *

from scipy import *
from scipy import integrate
from scipy.integrate import ode

In [2]:
#Importa Parametros de control

f = open ('parrov_bluerov.txt','r')
D={}
for i in f:
    [clave, valor]= i.split("=")
    D[clave] = float(valor[:-2])
f.close()
#Se asignan las variables... esto se hace para que sea sencillo describir mas adelante las matrices .... Ya estan los C+ y C- ??
[m,xg,yg,zg,xb,yb,zb,Ix,Iy,Iz,Ixy,Ixz,Iyz,W,B,Xu,Yv,Zw,Kp,Mq,Nr,Xudot,Yvdot,Zwdot,Kpdot,Mqdot,Nrdot,Xuu,Yvv,Zww,Kpp,Mqq,Nrr,lx1,ly1,lz1,lx2,ly2,lz2,lx3,ly3,lz3,lx4,ly4,lz4,lx5,ly5,lz5,lx6,ly6,lz6,Cmas1,Cmas2,Cmas3,Cmas4,Cmas5,Cmas6,Cmenos1,Cmenos2,Cmenos3,Cmenos4,Cmenos5,Cmenos6 ]   =   [D['m'],D['xg'],D['yg'],D['zg'],D['xb'],D['yb'],D['zb'],D['Ix'],D['Iy'],D['Iz'],D['Ixy'],D['Ixz'],D['Iyz'],D['W'],D['B'],D['Xu'],D['Yv'],D['Zw'],D['Kp'],D['Mq'],D['Nr'],D['Xudot'],D['Yvdot'],D['Zwdot'],D['Kpdot'],D['Mqdot'],D['Nrdot'],D['Xuu'],D['Yvv'],D['Zww'],D['Kpp'],D['Mqq'],D['Nrr'],D['lx1'],D['ly1'],D['lz1'],D['lx2'],D['ly2'],D['lz2'],D['lx3'],D['ly3'],D['lz3'],D['lx4'],D['ly4'],D['lz4'],D['lx5'],D['ly5'],D['lz5'],D['lx6'],D['ly6'],D['lz6'],0.000333,0.000333,0.000224,0.000241,0.000241,0.000243,0.000368,0.000368,0.000227,0.000175,0.000175,0.000229]




In [3]:
#Se crean las variables simbolicas dependientes e independientes

#Entradas para el modelo dinamico .... Torques aplicados por los propulsores
u1 = symbols('u1')
u2 = symbols('u2')
u3 = symbols('u3')
u4 = symbols('u4')
u5 = symbols('u5')
u6 = symbols('u6')
U= np.array([u1,u2,u3,u4,u5,u6]) # Se define el vector de entradas

#Salidas de mi modelo dinamico... o velocidades en el marco de referencia local
u = symbols('u')
v = symbols('v')
w = symbols('w')
p = symbols('p')
q = symbols('q')
r = symbols('r')
V = np.array([u,v,w,p,q,r])
t = symbols("t") #Tiempo

#Salidas de posiciones lineales y angulares del marco de referencia inercial en funcion a las velocidades del marco local
x = symbols('x')
y = symbols('y')
z = symbols('z')
theta = symbols('theta')
phi = symbols('phi')
psi = symbols('psi')
n = np.array([x,y,z,theta,phi,psi])



In [4]:
#Se construye la matriz J1 y J2 y finalmente J2
from sympy import *

J1 = np.array([[cos(theta)*cos(psi),(-cos(phi)*sin(psi))+(sin(phi)*sin(theta)*cos(psi)), (sin(phi)*sin(psi))+ (cos(phi)*sin(theta)*cos(psi))],
               [cos(theta)*sin(psi),(cos(phi)*cos(psi))+(sin(phi)*sin(theta)*sin(psi)) ,(-sin(phi)*cos(psi))+ (cos(phi)*sin(theta)*sin(psi))],
               [   -sin(theta)     ,            cos(theta)*sin(phi)                    ,                   cos(theta)*cos(phi)              ]])
J2 = np.array([[     1.1     ,  sin(phi)*sin(theta)/cos(theta)  ,  cos(phi)*sin(theta)/cos(theta)  ],
               [     0.0     ,             cos(phi)             ,            -sin(phi)             ],
               [     0.0     ,        sin(phi)/cos(theta)       ,               1.1                ]])
zeros = np.zeros((3,3))
J = np.concatenate((np.concatenate((J1, zeros), axis=1), np.concatenate((zeros, J2), axis=1)), axis=0)
print("J1 = \n{}".format(J1))
print("J2 = \n{}".format(J2))
print("J = \n{}".format(J))


J1 = 
[[cos(psi)*cos(theta) sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi)
  sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi)]
 [sin(psi)*cos(theta) sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi)
  -sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi)]
 [-sin(theta) sin(phi)*cos(theta) cos(phi)*cos(theta)]]
J2 = 
[[1.1 sin(phi)*sin(theta)/cos(theta) sin(theta)*cos(phi)/cos(theta)]
 [0.0 cos(phi) -sin(phi)]
 [0.0 sin(phi)/cos(theta) 1.1]]
J = 
[[cos(psi)*cos(theta) sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi)
  sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi) 0.0 0.0 0.0]
 [sin(psi)*cos(theta) sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi)
  -sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi) 0.0 0.0 0.0]
 [-sin(theta) sin(phi)*cos(theta) cos(phi)*cos(theta) 0.0 0.0 0.0]
 [0.0 0.0 0.0 1.1 sin(phi)*sin(theta)/cos(theta)
  sin(theta)*cos(phi)/cos(theta)]
 [0.0 0.0 0.0 0.0 cos(phi) -sin(phi)]
 [0.0 0.0 0.0 0.0 sin(phi)/cos(theta) 1.1]]


In [5]:
#Matriz M
#Matriz M_RB                     
M_RB = np.array([[     m        ,        0      ,        0      ,      0        ,     m*zg     ,       -m*yg    ],
                 [     0        ,        m      ,        0      ,    -m*zg      ,       0      ,        m*xg    ],
                 [     0        ,        0      ,        m      ,     m*yg      ,    -m*xg     ,          0     ],
                 [     0        ,      -m*zg    ,       m*yg    ,     -Ix       ,    -Ixy      ,        -Ixz    ],
                 [    m*zg      ,        0      ,      -m*xg    ,     -Ixy      ,     Iy       ,        -Iyz    ],
                 [   -m*yg      ,       m*xg    ,        0      ,     -Ixz      ,     -Iyz     ,         Iz     ]])
#Matriz M_A
M_A = np.array([Xudot,Yvdot,Zwdot,Kpdot,Mqdot,Nrdot]) * np.eye(6) # MUltiplica vector valores con matris identidad 6x6

M= M_RB + M_A

print("M_RB = \n{}".format(M_RB))
print("M_A = \n{}".format(M_A))
print("M = \n{}".format(M))

M_RB = 
[[ 7.31  0.    0.    0.    0.   -0.  ]
 [ 0.    7.31  0.   -0.    0.    0.  ]
 [ 0.    0.    7.31  0.   -0.    0.  ]
 [ 0.   -0.    0.   -0.16 -0.   -0.  ]
 [ 0.    0.   -0.   -0.    0.16 -0.  ]
 [-0.    0.    0.   -0.   -0.    0.16]]
M_A = 
[[ -5.5   -0.    -0.    -0.    -0.    -0.  ]
 [ -0.   -12.7   -0.    -0.    -0.    -0.  ]
 [ -0.    -0.   -14.57  -0.    -0.    -0.  ]
 [ -0.    -0.    -0.    -0.12  -0.    -0.  ]
 [ -0.    -0.    -0.    -0.    -0.12  -0.  ]
 [ -0.    -0.    -0.    -0.    -0.    -0.12]]
M = 
[[ 1.81  0.    0.    0.    0.   -0.  ]
 [ 0.   -5.39  0.   -0.    0.    0.  ]
 [ 0.    0.   -7.26  0.   -0.    0.  ]
 [ 0.   -0.    0.   -0.28 -0.   -0.  ]
 [ 0.    0.   -0.   -0.    0.04 -0.  ]
 [-0.    0.    0.   -0.   -0.    0.04]]


In [6]:
#Matriz C.... Matriz de fuerzas centrípeta y Coriolis
#Matriz C_RB
#Matriz C12 
C11 = np.eye(3)*0 #Crea matriz 3x3 de Ceros
C12 = np.array([[ m*(yg*q +zg*r), -m*(xg*q-w)  , -m*(xg*r+v)  ],
                [ -m*(yg*p+w)   , m*(zg*r+xg*p), -m*(yg*r-u)  ],
                [ -m*(zg*p-v)   , -m*(zg*q+u)  , m*(xg*p+yg*q)]])

C22 = np.array([[         0                 ,-Iyz*q-Ixz*p+Iz*r, Iyz*r+Ixy*p-Iy*q],
                [ Iyz*q+Ixz*p-+Iz*r,          0               ,-Ixz*r-Ixy*q+Iy*p],
                [-Iyz*r-Ixy*p+Iy*q , Ixz*r+Ixy*q-Iy*p, 0                  ]])
C21 = np.transpose(C12) # Transpuesta de C12
C_RB = np.concatenate((np.concatenate((C11, C12), axis=1), np.concatenate((C21, C22), axis=1)), axis=0)



CA = np.array([[     0        ,        0      ,        0      ,      0        , -Zwdot*w  ,     Yvdot*v ],
               [     0        ,        0      ,        0      ,  Zwdot*w   ,       0      ,    -Xudot*u ],
               [     0        ,        0      ,        0      , -Yvdot*v   ,  Xudot*u  ,          0     ],
               [      0       , -Zwdot*w   ,    Yvdot*v ,      0        , -Nrdot*r  ,     Mqdot*q ],
               [  Zwdot*w  ,        0      ,   -Xudot*u ,  Nrdot*r   ,       0      ,    -Kpdot*p ],
               [ -Yvdot*v  ,  Xudot*u   ,          0    , -Mqdot*q   ,  Kpdot*p  ,          0     ]])

C = C_RB + CA
print("C_RB = \n{}".format(C_RB))
print("CA = \n{}".format(CA))
print("C = \n{}".format(C))


C_RB = 
[[0.0 0.0 0.0 0 7.31*w -7.31*v]
 [0.0 0.0 0.0 -7.31*w 0 7.31*u]
 [0.0 0.0 0.0 7.31*v -7.31*u 0]
 [0 -7.31*w 7.31*v 0 0.16*r -0.16*q]
 [7.31*w 0 -7.31*u -0.16*r 0 0.16*p]
 [-7.31*v 7.31*u 0 0.16*q -0.16*p 0]]
CA = 
[[0 0 0 0 14.57*w -12.7*v]
 [0 0 0 -14.57*w 0 5.5*u]
 [0 0 0 12.7*v -5.5*u 0]
 [0 14.57*w -12.7*v 0 0.12*r -0.12*q]
 [-14.57*w 0 5.5*u -0.12*r 0 0.12*p]
 [12.7*v -5.5*u 0 0.12*q -0.12*p 0]]
C = 
[[0.0 0.0 0.0 0 21.88*w -20.01*v]
 [0.0 0.0 0.0 -21.88*w 0 12.81*u]
 [0.0 0.0 0.0 20.01*v -12.81*u 0]
 [0 7.26*w -5.39*v 0 0.28*r -0.28*q]
 [-7.26*w 0 -1.81*u -0.28*r 0 0.28*p]
 [5.39*v 1.81*u 0 0.28*q -0.28*p 0]]


In [7]:
#Matriz de amortiguamiento hidrodinamico
DL = np.array([Xu,Yv,Zw,Kp,Mq,Nr]) * np.eye(6)
DQ = np.array([Xuu*u,Yvv*v,Zww*w,Kpp*p,Mqq*q,Nrr*r]) * np.eye(6)
D = DL + DQ
print("DL = \n{}".format(DL))
print("DQ = \n{}".format(DQ))
print("C = \n{}".format(C))


DL = 
[[-4.03 -0.   -0.   -0.   -0.   -0.  ]
 [-0.   -6.22 -0.   -0.   -0.   -0.  ]
 [-0.   -0.   -5.18 -0.   -0.   -0.  ]
 [-0.   -0.   -0.   -0.07 -0.   -0.  ]
 [-0.   -0.   -0.   -0.   -0.07 -0.  ]
 [-0.   -0.   -0.   -0.   -0.   -0.07]]
DQ = 
[[-18.18*u 0 0 0 0 0]
 [0 -21.66*v 0 0 0 0]
 [0 0 -36.99*w 0 0 0]
 [0 0 0 -1.55*p 0 0]
 [0 0 0 0 -1.55*q 0]
 [0 0 0 0 0 -1.55*r]]
C = 
[[0.0 0.0 0.0 0 21.88*w -20.01*v]
 [0.0 0.0 0.0 -21.88*w 0 12.81*u]
 [0.0 0.0 0.0 20.01*v -12.81*u 0]
 [0 7.26*w -5.39*v 0 0.28*r -0.28*q]
 [-7.26*w 0 -1.81*u -0.28*r 0 0.28*p]
 [5.39*v 1.81*u 0 0.28*q -0.28*p 0]]


In [8]:
#Matriz de configuracion de los propulsores T
T= np.array([[     0        ,        0      ,        0      ,      1        ,       1     ,          0      ],
             [     0        ,        m      ,        0      ,      0        ,       0     ,          1      ],
             [    -1        ,        -1     ,        -1     ,      0        ,       0     ,          0      ],
             [   -ly1       ,      -ly2     ,      -ly3     ,      0        ,       0     ,        -lz6     ],
             [    lx1       ,        lx2    ,       lx3     ,     lz4       ,      lz5    ,          0      ],
             [     0        ,        0      ,        0      ,    -ly4       ,     -ly5    ,         lx6     ]])
Cmas = np.array([Cmas1,Cmas2,Cmas3,Cmas4,Cmas5,Cmas6]) * np.eye(6)
TC = np.dot(T,Cmas)
print("T = \n{}".format(T))
print("C+ = \n{}".format(Cmas))
print("TC+ = \n{}".format(TC))

print("C = \n{}".format(C))


T = 
[[ 0.00e+00  0.00e+00  0.00e+00  1.00e+00  1.00e+00  0.00e+00]
 [ 0.00e+00  7.31e+00  0.00e+00  0.00e+00  0.00e+00  1.00e+00]
 [-1.00e+00 -1.00e+00 -1.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 1.11e-04 -1.11e-04 -0.00e+00  0.00e+00  0.00e+00 -9.50e-05]
 [ 1.53e-04  1.53e-04 -1.99e-04 -1.40e-05 -1.40e-05  0.00e+00]
 [ 0.00e+00  0.00e+00  0.00e+00 -1.11e-04  1.11e-04  8.00e-06]]
C+ = 
[[0.000333 0.       0.       0.       0.       0.      ]
 [0.       0.000333 0.       0.       0.       0.      ]
 [0.       0.       0.000224 0.       0.       0.      ]
 [0.       0.       0.       0.000241 0.       0.      ]
 [0.       0.       0.       0.       0.000241 0.      ]
 [0.       0.       0.       0.       0.       0.000243]]
TC+ = 
[[ 0.00000e+00  0.00000e+00  0.00000e+00  2.41000e-04  2.41000e-04
   0.00000e+00]
 [ 0.00000e+00  2.43423e-03  0.00000e+00  0.00000e+00  0.00000e+00
   2.43000e-04]
 [-3.33000e-04 -3.33000e-04 -2.24000e-04  0.00000e+00  0.00000e+00
   0.00000e+00]
 [ 3.69630e

In [94]:
#Vector de fuerza gravitacional y de empuje

g = [                         (W-B)*sin(theta)                             ,    
                         -(W-B)*cos(theta)*sin(phi)                        ,    
                         -(W-B)*cos(theta)*cos(phi)                        ,     
     -((yg*W)-(yb*B))*cos(theta)*cos(phi) + (zg*w-zb*B)*cos(theta)*sin(phi),
           -((zg*W)-(zb*B))*sin(theta) + (xg*W-xb*B)*cos(theta)*cos(phi)   ,
       -((xg*W)-(xb*B))*cos(theta)*sin(phi) - (yg*W-yb*B)*sin(theta)         ]
print(g)
print("C = \n{}".format(C))


[-6.67*sin(theta), 6.67*sin(phi)*cos(theta), 6.67*cos(phi)*cos(theta), -0.0148922*sin(phi)*cos(theta), 0.0148922*sin(theta), 0]
C = 
[[0.0 0.0 0.0 0 21.88*w -20.01*v]
 [0.0 0.0 0.0 -21.88*w 0 12.81*u]
 [0.0 0.0 0.0 20.01*v -12.81*u 0]
 [0 7.26*w -5.39*v 0 0.28*r -0.28*q]
 [-7.26*w 0 -1.81*u -0.28*r 0 0.28*p]
 [5.39*v 1.81*u 0 0.28*q -0.28*p 0]]


In [95]:
#Se replantea la ecuacion diferencial despejando Vdot
# INGRESO
D_ = np.dot(np.dot(  np.linalg.inv(M) ,  D ) ,V)  #M-1*D*V
T_ = np.dot(np.dot(  np.linalg.inv(M) ,  TC) ,U) #M-1*TC+*V
print("M = \n{}".format( np.linalg.inv(M)))
print("C = \n{}".format(C))
print("V = \n{}".format(V))

C_ = np.dot(np.dot(  np.linalg.inv(M) ,  C ) ,V)  #M-1*C*V

Vdott = -C_ -D_ -g + T_
print("Vdot = \n{}".format(Vdott[4]))


#Se expresa ndot y de esta forma tenemos nuestro sistema de 12 ecuaciones diferenciales no lineales

ndott = np.dot(J,V) #Vector de velocidad en el marco de referencia
print(ndott)

M = 
[[ 0.55248619  0.          0.          0.          0.          0.        ]
 [-0.         -0.18552876 -0.         -0.         -0.         -0.        ]
 [-0.         -0.         -0.13774105 -0.         -0.         -0.        ]
 [ 0.          0.          0.         -3.57142857  0.          0.        ]
 [ 0.          0.          0.          0.         25.          0.        ]
 [ 0.          0.          0.          0.          0.         25.        ]]
C = 
[[0.0 0.0 0.0 0 21.88*w -20.01*v]
 [0.0 0.0 0.0 -21.88*w 0 12.81*u]
 [0.0 0.0 0.0 20.01*v -12.81*u 0]
 [0 7.26*w -5.39*v 0 0.28*r -0.28*q]
 [-7.26*w 0 -1.81*u -0.28*r 0 0.28*p]
 [5.39*v 1.81*u 0 0.28*q -0.28*p 0]]
V = 
[u v w p q r]
Vdot = 
-q*(-38.75*q - 1.75) + 226.75*u*w + 1.273725e-6*u1 + 1.273725e-6*u2 - 1.1144e-6*u3 - 8.435e-8*u4 - 8.435e-8*u5 - 0.0148922*sin(theta)
[u*cos(psi)*cos(theta) + v*(sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi)) + w*(sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi))
 u*sin(psi)*cos(theta) + v*(sin

In [105]:
def ResolvEcuations(Vdot,ndot,VSym,nSym,USym,V_ant,n_ant,U,dt): #Retorna acelaraciones locales y velocidades inerciales
    Values = []
    for i in range(6):
        Values.append((VSym.item(i),V_ant.item(i))) #Agrega tupla de (symbolo valor)   
        Values.append((nSym.item(i),n_ant.item(0))) #Agrega tupla de (symbolo valor)
        Values.append((USym.item(i),U.item(0)))     #Agrega tupla de (symbolo valor)
 
    _Vdot =[]
    _ndot =[]
    for i in range(6):
        _Vdot.append(Vdot[i].subs(Values))
        _ndot.append(ndot[i].subs(Values))
    if(n_ant[2] == 0):
        _Vdot[2]=0
    
    return [_Vdot*dt,_ndot*dt]
    
def IntegerEcuations(Vdot,ndot,VSym,nSym,USym,V_ant,n_ant,U,dt,T):
    VdotValue,ndotValue = ResolvEcuations(Vdott,ndott,V,n,U,ceros,ceros,ceros,1)

In [107]:
ceros = np.zeros(6)
for i in 
ResolvEcuations(Vdott,ndott,V,n,U,ceros,ceros,ceros,1)

[(u, 0.0), (x, 0.0), (u1, 0.0), (v, 0.0), (y, 0.0), (u2, 0.0), (w, 0.0), (z, 0.0), (u3, 0.0), (p, 0.0), (theta, 0.0), (u4, 0.0), (q, 0.0), (phi, 0.0), (u5, 0.0), (r, 0.0), (psi, 0.0), (u6, 0.0)]


In [ ]:

f_np = lambdify([u,v,w,p,q,r], Vdott[0])

y0 = 0
xp = np.linspace(0, 1.9, 100)
X = [xp,xp,xp,xp,xp,xp]
# Calculando la solución numerica para los valores de y0 y xp
yp = integrate.odeint(f_np, [0,0,0,0,0,0], X)

In [ ]:
V0 = [0,0,0,0,0,0]
Up = [x,x,x,x,x,x]
print(Vdott)
#yn = integrate.odeint(f_np, V0, Up)